In [9]:
import pandas as pd
import numpy as np
from random import sample
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import pickle

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithzerostopredict.csv')
topredict= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/topredict.csv')
df_withoutzero= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithoutzeros.csv')

***df has all the topredict values entered as 0 for the Book-Rating column.***</br>
***topredict has all the values that we want to predict.***</br>
***df_withoutzero has all the topredict all the values including the values in the test dataset.***



In [12]:
df['User-ID'].nunique()

77805

In [13]:
topredict['User-ID'].nunique()

19935

##**500 users that rate the most**

In [14]:
df_num_rating= df.groupby('User-ID').agg(Number_of_ratings=('Book-Rating','count')).reset_index()
# Num ratings count

In [15]:
df_num_rating.sort_values(by=['Number_of_ratings'], ascending=False, inplace=True)
df_num_rating

,User-ID,Number_of_ratings
3160,11676,8524
27626,98391,5802
43027,153662,1969
52924,189835,1906
6510,23902,1395
...,...,...
23601,84129,1
46266,165812,1
9606,34231,1
46268,165826,1


In [16]:
top_users= df_num_rating['User-ID'][:500]
# top 500 users

##**To Predict User List and Batch Designation**

In [17]:
topredict_users= topredict['User-ID'].unique()
topredict_users

array([    17,     56,    114, ..., 278844, 278851, 278854])

In [18]:
len(topredict_users)

19935

**We divide the test into 20 folds because of memory limitations. We will predict the test values by loading in batches**

In [19]:
dict_batch= {}
for i in range(0,len(topredict_users),1000):
  dict_batch['fold_{}'.format(i//1000)]= topredict_users[i:i+1000]

In [20]:
dict_batch #fold_0 to fold_19#

{'fold_0': array([   17,    56,   114,   160,   183,   242,   243,   254,   272,
          289,   300,   362,   383,   388,   392,   408,   424,   440,
          441,   444,   446,   472,   476,   487,   503,   505,   507,
          566,   619,   625,   626,   638,   640,   643,   651,   657,
          695,   709,   726,   741,   744,   746,   753,   776,   786,
          805,   819,   850,   853,   882,   885,   896,   899,   900,
          901,   914,   929,  1009,  1021,  1025,  1031,  1063,  1075,
         1083,  1096,  1113,  1116,  1129,  1131,  1155,  1161,  1167,
         1178,  1184,  1211,  1235,  1248,  1249,  1254,  1261,  1297,
         1343,  1376,  1409,  1412,  1421,  1424,  1435,  1436,  1467,
         1485,  1517,  1548,  1558,  1570,  1585,  1596,  1597,  1619,
         1652,  1660,  1667,  1674,  1688,  1696,  1733,  1790,  1791,
         1797,  1805,  1830,  1848,  1903,  1990,  2009,  2010,  2024,
         2030,  2033,  2041,  2046,  2084,  2090,  2103,  2106,  21

In [21]:
# this is the matrix factorization function from the MF notebook. 
def MF(M,k,max_it,lambd,mu):
    n=M.size()[0]
    m= M.size()[1]
    nonzero=len(M.nonzero())
    index= M.nonzero().split(1, dim=1)
    #param=torch.rand(n*k+k*m,dtype=float,requires_grad=True)
    param1=torch.rand((n,k),dtype=torch.float,requires_grad=True)
    param2=torch.rand((k,m),dtype=torch.float,requires_grad=True)

    opt1= torch.optim.Adam([param1],lr=0.1)
    opt2= torch.optim.Adam([param2],lr=0.1)
    
    #scheduler1= ReduceLROnPlateau(opt1, 'min') 
    #scheduler2 = ReduceLROnPlateau(opt2, 'min')
    

    #def get_loss(params,params_hat):
        #return torch.sum(torch.square(params- params_hat))

    def run_iterations(max_it):
        loss_record=[]
        converged=False
        for it in tqdm(range(max_it)):
            if it%2==0:
                opt1.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt1.step()
                #scheduler1.step(loss)
            else:
                opt2.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt2.step()
                #scheduler2.step(loss)
        display(loss_record)
        return torch.matmul(param1,param2)
    return run_iterations(max_it)

**The following loop loads the ith batch of the train dataset and constructs a matrix using the ith batch and the subset of the train dataset containing the 500 top users. Then it computes the matrix factorization prediction on the ith batch. It calculates the MSE for each batch in each iteration.**

In [22]:
mse_batch=[] # this will collect the batch MSE
indices_batch=[] # this will collect the indices of the test dataset in each batch
for i in range(len(dict_batch)): #length of dict_batch is 20 because we have 20 folds
  df_batch= topredict[topredict['User-ID'].isin(dict_batch['fold_{}'.format(i)])].reset_index(drop=True)
  df_matrix= df[df['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))].reset_index(drop=True) #combines data from 500 top users and the test batch
  mat= df_matrix.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0) # matrix using pivot
  matrix= torch.tensor(mat.values)/10 #converts to tensor and scales the values by 1/10
  dict_user= dict(zip(sorted(set(df_matrix['User-ID'])),range(len(sorted(set(df_matrix['User-ID']))))))
  dict_book= dict(zip(sorted(set(df_matrix['ISBN'])),range(len(sorted(set(df_matrix['ISBN']))))))
  index1=[] #this will collect the row index
  index2=[] #this will collect the column index
  for j in range(len(df_batch)):
    index1.append([dict_user[df_batch['User-ID'][j]]])
    index2.append([dict_book[df_batch['ISBN'][j]]])
  indices_topred = (torch.tensor(index1),torch.tensor(index1)) #this contains the indices from the matrix whose values we want to predict
  # this is a bit peculiar but torch tensors work this way
  df_actual= df_withoutzero[df_withoutzero['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))]
  # df_actual is the matrix with the ratings of the test dataset included
  # we need this to calculate MSE
  actual_matrix= df_actual.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0)
  actual_matrix= torch.tensor(actual_matrix.values)/10 # convert to tensor and scale like above

  mse_batch.append(100*torch.sum(torch.square(MF(matrix,10,1500,0.1,0.1)[indices_topred]- actual_matrix[indices_topred])))
  # we multiply by 10^2 because we scaled by 1/10 and squared
  # the values 25,0.1,0.1 come from the MF validation notebook where we performed validation for parameter tuning 
  indices_batch.append(len(indices_topred[0]))
  display(mse_batch[-1])






100%|██████████| 1500/1500 [09:48<00:00,  2.55it/s]


[417782.43285339617,
 250582.62210716488,
 141137.4343929216,
 82179.77189549133,
 45896.38928847458,
 35388.18536310081,
 27320.587131621127,
 32378.061310013443,
 31360.170629022905,
 38733.55011540025,
 37562.398396559234,
 43346.66332634709,
 40615.79756844395,
 44327.12745439111,
 40469.18576982042,
 42563.03797111553,
 38332.0353095903,
 39358.50428402485,
 35345.605278681374,
 35730.10088893239,
 32266.866859994203,
 32293.445845016373,
 29489.485416968077,
 29335.811821277006,
 27154.775648644652,
 26929.330505606995,
 25262.32212964457,
 25030.26318832387,
 23746.67138239398,
 23544.227793499118,
 22519.93562041162,
 22361.832938580716,
 21492.87742148172,
 21377.053042455642,
 20585.16487042119,
 20497.80664966951,
 19731.01870112797,
 19653.552461552834,
 18884.223225768907,
 18800.71516142553,
 18023.799237498562,
 17925.39078930138,
 17153.344734120878,
 17038.664109515757,
 16292.505433280403,
 16165.253478857694,
 15465.909394807191,
 15331.587884881548,
 14694.010379431

tensor(116041.1186, dtype=torch.float64, grad_fn=<MulBackward0>)

100%|██████████| 1500/1500 [10:18<00:00,  2.43it/s]


[368914.71290312085,
 215229.67933080636,
 120960.36725807172,
 69927.68514811122,
 40257.25153110045,
 33243.153380506454,
 26834.617473941944,
 32897.008436964315,
 31530.854366909516,
 38852.10481815859,
 36873.059479412725,
 42295.792721644066,
 38920.53042803603,
 42275.76365136951,
 38097.351689822244,
 39941.06190616318,
 35671.606383419465,
 36559.82466876387,
 32685.308143857706,
 33016.40055897054,
 29773.833432411324,
 29803.92307512089,
 27244.85760171875,
 27132.879562944465,
 25191.206951182252,
 25031.04461818519,
 23577.414334275454,
 23415.781500100322,
 22300.442062992162,
 22150.2463664787,
 21235.890633637522,
 21090.990579206442,
 20273.304136817576,
 20123.07637964121,
 19338.772562234615,
 19177.655401895972,
 18401.58536628548,
 18231.54890422956,
 17465.537959089717,
 17293.55601254625,
 16552.531729084527,
 16386.454067610917,
 15686.915011175524,
 15532.241056954426,
 14884.959033211462,
 14743.377174123842,
 14150.863892569152,
 14020.47674577009,
 13478.921

...

[401761.1314476526,
 237749.71927934932,
 133695.37392332556,
 76590.30284956995,
 43160.831615718314,
 33689.07215703108,
 26767.934624305766,
 32412.79151753307,
 31545.70752532589,
 39238.64765606704,
 37686.44299062689,
 43630.94130098277,
 40283.679572509245,
 44071.225123548465,
 39559.003947326244,
 41695.954729105084,
 36901.08386095674,
 37968.464100494595,
 33559.09941095246,
 33999.68944466178,
 30343.846797497794,
 30450.66404696236,
 27656.188047128475,
 27608.00590881005,
 25597.755169762117,
 25494.146005471117,
 24085.53405761451,
 23972.87647750406,
 22949.81019877648,
 22840.685340916643,
 22010.22123162952,
 21899.52188438607,
 21125.83356216343,
 21003.868725213528,
 20216.692020399198,
 20077.485214110897,
 19260.94268552921,
 19105.309466688086,
 18277.545923963607,
 18112.244491429017,
 17305.214283400204,
 17139.563951393986,
 16383.562387982258,
 16225.483199401158,
 15540.020578298147,
 15393.633781027198,
 14784.729889793703,
 14649.937795936716,
 14112.33892

tensor(24445.1088, dtype=torch.float64, grad_fn=<MulBackward0>)

100%|██████████| 1500/1500 [10:59<00:00,  2.28it/s]


[378734.99061484006,
 221388.52692496445,
 124093.94210618219,
 70815.86498528402,
 40378.8624747237,
 32681.80815242092,
 26509.20309608042,
 32767.118402832297,
 31707.343744890004,
 39465.03434359249,
 37541.1864334576,
 43341.15082263503,
 39769.40307272819,
 43365.28346126112,
 38844.56741267051,
 40807.676487081044,
 36174.93352945733,
 37106.78687794361,
 32949.54586528428,
 33294.989044265356,
 29899.02583224651,
 29942.47653323268,
 27348.301426629594,
 27252.227339181565,
 25344.79631896983,
 25192.02177776872,
 23789.978682259614,
 23618.115981545547,
 22540.35042673876,
 22363.577217308448,
 21465.273943353444,
 21286.216784028464,
 20470.10679847123,
 20287.454175717477,
 19500.24676481405,
 19313.937649635336,
 18536.03060226354,
 18349.505366858983,
 17583.14197695926,
 17402.472563177987,
 16660.77947414564,
 16491.952183421592,
 15790.170379045196,
 15636.971183419446,
 14986.907374002429,
 14850.253917811538,
 14257.426480344293,
 14135.463298941413,
 13598.1400788083

tensor(14735.8750, dtype=torch.float64, grad_fn=<MulBackward0>)

In [23]:
with open("/content/drive/MyDrive/Colab Notebooks/mse_batch", "wb") as fp:   #Pickling
  pickle.dump(mse_batch, fp)

with open("/content/drive/MyDrive/Colab Notebooks/indices_num_batch", "wb") as fp:   #Pickling
  pickle.dump(indices_batch, fp)

In [34]:
mse= sum(mse_batch)/sum(indices_batch)

In [35]:
mse

tensor(7.8160, dtype=torch.float64, grad_fn=<DivBackward0>)

In [36]:
rmse= np.sqrt(mse.detach())
rmse

tensor(2.7957, dtype=torch.float64)